# Polymarket RL Trading Bot — Model Training Architecture

## Overview

This project trains a reinforcement learning agent to trade 5-minute BTC Up/Down binary markets on Polymarket. The agent observes the live order book for both tokens alongside BTC price features and learns to take directional positions in prediction markets.

---

## Data Pipeline

### Live Ingest (Operational, not used for training)
- Subscribes to the Polymarket WebSocket feed
- Receives real-time `BookSnapshot` and `TradeEvent` messages
- Dumps events onto a Redis stream
- A persistence component reads the stream, normalizes to a protobuf schema, and writes to `.parquet` files under `data/book_snapshots/` and `data/trade_events/`
- **Token labels in live data: `Yes` / `No`**

### Historical Data — Telonex (`telonex_pipeline.py`)
- Fetches historical order book snapshots (5-level depth) for all BTC 5-minute markets from the Telonex API
- Coverage: **Feb 12–25, 2026** (~3,732 markets)
- Raw files downloaded to `datasets/telonex_raw/`, normalized to `data/telonex_book_snapshots/`
- Normalization handles: bid sort (ascending → descending), market window filtering (open_ms to open_ms + 300,000ms), filename mapping (asset_id → slug via Gamma API lookup)
- **Token labels in Telonex data: `Up` / `Down`**
- ⚠️ Schema match against live data has **not been formally validated** column-by-column

### BTC Price Data (`telonex_btc_pipeline.py` / `btc_pipeline.py`)
- Source: Binance BTC/USDT quotes via Telonex (100ms depth updates)
- Resampled to **1-second bars**
- Derived features: `log_return`, `ret_5s`, `ret_15s`, `ret_60s`, `rvol_30s`, `rvol_60s`, `rvol_300s`
- Output: `data/btc_quotes/btcusdt_quotes.parquet` (1.12M rows, 13 days)

### Market Resolutions (`market_analysis.py` — `ResolutionStore`)
- Fetches resolution outcomes from Gamma API: `https://gamma-api.polymarket.com/markets/slug/{slug}`
- Cached to `data/resolutions.json`
- Schema: `{token_id: 1.0}` for winner, `{token_id: 0.0}` for loser
- Originally populated for live-capture slugs; **Telonex slug compatibility not yet confirmed**

### Chainlink Oracle Data
- **Pending.** Would provide the oracle price at each point in time, which is what determines market resolution. Key feature for the CEX-oracle divergence signal.

---

## Gymnasium Environment (`polymarket_env.py`)

### Episode Structure
- One episode = one resolved 5-minute market
- Steps through the order book at **100ms resampled intervals** (configurable via `RESAMPLE_MS`)
- Resampling: last value within each 100ms bar, forward-filled, backward-filled for gaps
- Result: **3,000 steps per episode** (uniform across all markets)

### Observation Space (61 dimensions)
| Group | Dims | Content |
|---|---|---|
| Yes book | 23 | mid, spread, imbalance, 5× bid (price, size), 5× ask (price, size) |
| No book | 23 | same as Yes |
| Position state | 6 | yes_position, no_position, yes_avg_cost, no_avg_cost, unrealized_pnl, capital_at_risk_pct |
| Time | 1 | time_remaining_pct (1.0 → 0.0) |
| BTC features | 8 | mid_norm, log_return, ret_5s, ret_15s, ret_60s, rvol_30s, rvol_60s, rvol_300s |

### Action Space — Discrete(6), Maskable
| Action | Description |
|---|---|
| 0 | Hold |
| 1 | Buy Yes small ($50) |
| 2 | Buy Yes large ($100) |
| 3 | Buy No small ($50) |
| 4 | Buy No large ($100) |
| 5 | Sell (flatten) |

**Action masking:** Buy actions masked at max position ($500 = 5% of $10k bankroll). Sell masked when flat. Uses `sb3-contrib` MaskablePPO interface.

### Position & Fill Model
- Fill model: **mid-price, immediate** (no slippage simulation)
- Cannot hold Yes and No simultaneously; buying one side auto-flattens the other at current mid
- Min order: $5. Small order: $50. Large order: $100. Max position: $500.
- Average cost tracked via weighted average on add

### Reward
- **Step reward: 0** (no shaping — terminal only)
- **Terminal reward: realized PnL** for the episode
- ⚠️ Currently: positions held to expiry are settled at **final mid-price** (incorrect)
- Should be: settled at **1.0** (winner) or **0.0** (loser) per market resolution

---

## Prior Analysis

### Calibration Analysis
- Pulled live-captured book snapshots and trade events
- Joined with Gamma API resolutions
- Built calibration curves: implied probability vs actual win rate across price buckets and time windows
- **Result: No significant edge found** in tail mispricing hypothesis
- This prompted the pivot to RL as a framework for learning more complex conditional strategies

### Smart Money Observation
- Identified high-volume accounts with consistent positive returns
- Behavioral pattern: open both sides near 0.50 at market open, gradually build directional position, occasionally load cheap contracts on the losing side near expiry
- Interpreted as: continuously updated internal probability estimate, trading spread between their model and market price throughout the full 5-minute window
- This behavioral pattern is consistent with an RL-style EV maximization strategy

---

## Training Setup (Planned)

- Algorithm: **MaskablePPO** (`sb3-contrib`)
- Framework: **Stable Baselines 3**
- Training data: 3,732 Telonex markets (Feb 12–25, 2026)
- BTC features joined at 1s granularity aligned to each 100ms step
- Reward shaping strategy: **TBD** — terminal-only reward is likely too sparse for efficient learning

# Sanity Runthrough

1. Check live ingest data for schema:

In [9]:
"""
Schema comparison: live ingest vs Telonex historical data.

Usage:
    python schema_comparison.py

Paths are configured at the top. The script will auto-select a slug that
exists in both directories. Override SLUG to force a specific market.
"""

from pathlib import Path
import pandas as pd
import numpy as np

# ---------------------------------------------------------------------------
# Config — adjust paths to match your setup
# ---------------------------------------------------------------------------

LIVE_DIR     = Path("data/book_snapshots")
HIST_DIR     = Path("data/telonex_book_snapshots")
SLUG         = "btc-updown-5m-1771348500"         # Set to e.g. "btc-updown-5m-1771348200" to force
N_LEVELS     = 5             # Number of book levels to compare (Telonex only has 5)
TIMESTAMP_COL = "exchange_timestamp"

# ---------------------------------------------------------------------------
# Helpers
# ---------------------------------------------------------------------------

def load(path: Path) -> pd.DataFrame:
    df = pd.read_parquet(path)
    df = df.sort_values(TIMESTAMP_COL).reset_index(drop=True)
    return df

def print_schema(df: pd.DataFrame, label: str):
    print(f"\n{'='*60}")
    print(f"  SCHEMA: {label}")
    print(f"{'='*60}")
    print(f"  Rows: {len(df):,}")
    print(f"  Columns ({len(df.columns)}):")
    for col in df.columns:
        print(f"    {col:<30} {str(df[col].dtype):<12}  "
              f"sample={repr(df[col].iloc[0]) if len(df) else 'N/A'}")

def find_common_slug() -> str | None:
    live_slugs = {p.stem for p in LIVE_DIR.glob("btc-updown-5m-*.parquet")}
    hist_slugs = {p.stem for p in HIST_DIR.glob("btc-updown-5m-*.parquet")}
    common = live_slugs & hist_slugs
    if not common:
        return None
    return sorted(common)[0]

# ---------------------------------------------------------------------------
# Main
# ---------------------------------------------------------------------------

def main():
    slug = SLUG or find_common_slug()
    if slug is None:
        print("ERROR: No slug found in both directories.")
        print(f"  Live slugs in {LIVE_DIR}: {len(list(LIVE_DIR.glob('*.parquet')))}")
        print(f"  Hist slugs in {HIST_DIR}: {len(list(HIST_DIR.glob('*.parquet')))}")
        return

    live_path = LIVE_DIR / f"{slug}.parquet"
    hist_path = HIST_DIR / f"{slug}.parquet"

    if not live_path.exists():
        print(f"ERROR: Live file not found: {live_path}")
        return
    if not hist_path.exists():
        print(f"ERROR: Historical file not found: {hist_path}")
        return

    print(f"\nComparing slug: {slug}")
    print(f"  Live: {live_path}")
    print(f"  Hist: {hist_path}")

    live = load(live_path)
    hist = load(hist_path)

    # -----------------------------------------------------------------------
    # 1. Schemas
    # -----------------------------------------------------------------------
    print_schema(live, f"LIVE  ({live_path})")
    print_schema(hist, f"HIST  ({hist_path})")

    # -----------------------------------------------------------------------
    # 2. Column-level diff
    # -----------------------------------------------------------------------
    live_cols = set(live.columns)
    hist_cols = set(hist.columns)

    print(f"\n{'='*60}")
    print("  COLUMN DIFF")
    print(f"{'='*60}")

    only_live = live_cols - hist_cols
    only_hist = hist_cols - live_cols
    shared    = live_cols & hist_cols

    print(f"\n  Only in LIVE  ({len(only_live)}): {sorted(only_live)}")
    print(f"  Only in HIST  ({len(only_hist)}): {sorted(only_hist)}")
    print(f"  Shared        ({len(shared)}): {sorted(shared)}")

    # Check dtype agreement on shared columns
    print(f"\n  Dtype mismatches on shared columns:")
    mismatched_dtypes = []
    for col in sorted(shared):
        ld = str(live[col].dtype)
        hd = str(hist[col].dtype)
        if ld != hd:
            mismatched_dtypes.append((col, ld, hd))
            print(f"    {col:<30} live={ld}  hist={hd}")
    if not mismatched_dtypes:
        print("    None — all shared columns have matching dtypes ✓")

    # -----------------------------------------------------------------------
    # 3. Align on timestamp and token — compare shared numeric columns
    # -----------------------------------------------------------------------
    # Telonex data has both Up and Down rows in the same file.
    # Live data files contain rows for a single asset_id (one token per file).
    # We need to split Telonex by token and compare each to the live file.

    # Determine which token the live file contains
    live_asset_ids = live["asset_id"].unique() if "asset_id" in live.columns else []
    hist_asset_ids = hist["asset_id"].unique() if "asset_id" in hist.columns else []

    print(f"\n{'='*60}")
    print("  TIMESTAMP ALIGNMENT & DELTA ANALYSIS")
    print(f"{'='*60}")
    print(f"\n  Live asset_ids : {list(live_asset_ids)}")
    print(f"  Hist asset_ids : {list(hist_asset_ids)}")

    # Determine numeric columns to diff — book levels up to N_LEVELS, plus derived
    numeric_cols_to_compare = (
        ["mid_price", "spread", "book_imbalance"]
        + [f"bid_price_{i}" for i in range(1, N_LEVELS + 1)]
        + [f"bid_size_{i}"  for i in range(1, N_LEVELS + 1)]
        + [f"ask_price_{i}" for i in range(1, N_LEVELS + 1)]
        + [f"ask_size_{i}"  for i in range(1, N_LEVELS + 1)]
    )
    # Keep only columns present in both
    numeric_cols_to_compare = [c for c in numeric_cols_to_compare
                                if c in live_cols and c in hist_cols]

    print(f"\n  Numeric columns being compared ({len(numeric_cols_to_compare)}): "
          f"{numeric_cols_to_compare}")

    # Split Telonex by token label if multi-token
    hist_tokens = {}
    if "token_label" in hist.columns:
        for label, group in hist.groupby("token_label"):
            hist_tokens[label] = group.reset_index(drop=True)
    else:
        # No token_label — treat as single group
        hist_tokens["(all)"] = hist

    # For each hist token, try to find a matching live subset by asset_id
    for token_label, hist_token_df in hist_tokens.items():
        print(f"\n  --- Token: {token_label} ---")

        # Match live rows by asset_id if possible
        if "asset_id" in hist_token_df.columns and "asset_id" in live.columns:
            token_asset_ids = hist_token_df["asset_id"].unique()
            live_subset = live[live["asset_id"].isin(token_asset_ids)].copy()
        else:
            live_subset = live.copy()

        print(f"  Live rows for this token : {len(live_subset):,}")
        print(f"  Hist rows for this token : {len(hist_token_df):,}")

        if live_subset.empty:
            print("  No matching live rows for this asset_id — skipping delta analysis.")
            continue

        # Align on timestamp
        live_idx = live_subset.set_index(TIMESTAMP_COL)
        hist_idx = hist_token_df.set_index(TIMESTAMP_COL)
        common_ts = live_idx.index.intersection(hist_idx.index)

        print(f"  Common timestamps        : {len(common_ts):,} "
              f"(live={len(live_idx)}, hist={len(hist_idx)})")

        if len(common_ts) == 0:
            print("  No overlapping timestamps — cannot compute deltas.")
            print(f"  Live ts range: {live_idx.index.min()} – {live_idx.index.max()}")
            print(f"  Hist ts range: {hist_idx.index.min()} – {hist_idx.index.max()}")
            continue

        live_aligned = live_idx.loc[common_ts, numeric_cols_to_compare]
        hist_aligned = hist_idx.loc[common_ts, numeric_cols_to_compare]

        # -----------------------------------------------------------------------
        # 4. Compute deltas
        # -----------------------------------------------------------------------
        delta = (live_aligned - hist_aligned).abs()

        print(f"\n  Delta summary (|live - hist|) across {len(common_ts):,} aligned rows:")
        print(f"\n  {'Column':<25} {'max_delta':>12} {'mean_delta':>12} {'nonzero_rows':>14}")
        print(f"  {'-'*65}")

        any_mismatch = False
        for col in numeric_cols_to_compare:
            if col not in delta.columns:
                continue
            col_delta = delta[col].dropna()
            max_d  = col_delta.max()
            mean_d = col_delta.mean()
            nz     = (col_delta > 0).sum()
            flag   = " ⚠" if nz > 0 else ""
            if nz > 0:
                any_mismatch = True
            print(f"  {col:<25} {max_d:>12.6f} {mean_d:>12.6f} {nz:>14,}{flag}")

        # -----------------------------------------------------------------------
        # 5. Show mismatched rows
        # -----------------------------------------------------------------------
        if any_mismatch:
            print(f"\n  Mismatched rows (delta > 0 in any column):")
            mismatch_mask = (delta > 0).any(axis=1)
            mismatch_ts   = delta[mismatch_mask].index

            print(f"  Total mismatched timestamps: {len(mismatch_ts):,}")
            print(f"  Showing first 10:\n")

            for ts in mismatch_ts[:10]:
                print(f"    timestamp={ts}")
                for col in numeric_cols_to_compare:
                    lv = live_aligned.loc[ts, col]
                    hv = hist_aligned.loc[ts, col]
                    d  = abs(lv - hv)
                    if d > 0:
                        print(f"      {col:<25} live={lv:.6f}  hist={hv:.6f}  delta={d:.6f}")
                print()
        else:
            print(f"\n  ✓ All {len(common_ts):,} aligned rows match exactly across "
                  f"{len(numeric_cols_to_compare)} columns.")

    print(f"\n{'='*60}")
    print("  SUMMARY")
    print(f"{'='*60}")
    print(f"  Slug           : {slug}")
    print(f"  Live columns   : {len(live_cols)}")
    print(f"  Hist columns   : {len(hist_cols)}")
    print(f"  Only in live   : {sorted(only_live)}")
    print(f"  Only in hist   : {sorted(only_hist)}")
    print(f"  Dtype clashes  : {len(mismatched_dtypes)}")
    print()


if __name__ == "__main__":
    main()


Comparing slug: btc-updown-5m-1771348500
  Live: data/book_snapshots/btc-updown-5m-1771348500.parquet
  Hist: data/telonex_book_snapshots/btc-updown-5m-1771348500.parquet

  SCHEMA: LIVE  (data/book_snapshots/btc-updown-5m-1771348500.parquet)
  Rows: 6,775
  Columns (50):
    exchange_timestamp             int64         sample=np.int64(1771348564459)
    received_timestamp             int64         sample=np.int64(1771348564481)
    market                         str           sample='0x41d6b8e6aa306f73a9e0d434a1381e38622b60df9cf9d8c9168c1ae04027d282'
    asset_id                       str           sample='55555626743217399864863921075568424822842661856294693148174408457001840592314'
    mid_price                      float64       sample=np.float64(0.4)
    spread                         float64       sample=np.float64(0.019999999999999962)
    book_imbalance                 float64       sample=np.float64(-0.09985952722900987)
    bid_price_1                    float64       sample

In [43]:
import pandas as pd
from pathlib import Path

slug = "btc-updown-5m-1771348800"  # use one of your matching slugs

live = pd.read_parquet(f"data/book_snapshots/{slug}.parquet")
tes = pd.read_parquet(f"data/trade_events/{slug}.parquet")
hist = pd.read_parquet(f"data/telonex_book_snapshots/{slug}.parquet")

# Check update intervals
live_up = live[live["asset_id"] == live["asset_id"].iloc[0]].sort_values("exchange_timestamp")
tes_up = tes[tes["asset_id"] == tes["asset_id"].iloc[0]].sort_values("exchange_timestamp")
hist_up = hist[hist["token_label"] == "Up"].sort_values("exchange_timestamp")

print("LIVE — interval between snapshots (ms):")
print(live_up["exchange_timestamp"].diff().describe())

print("\nHIST — interval between snapshots (ms):")
print(hist_up["exchange_timestamp"].diff().describe())

LIVE — interval between snapshots (ms):
count    4395.000000
mean       68.245279
std       116.302561
min         0.000000
25%        16.000000
50%        33.000000
75%        76.000000
max      2291.000000
Name: exchange_timestamp, dtype: float64

HIST — interval between snapshots (ms):
count    39169.000000
mean         7.655186
std         22.448675
min          1.000000
25%          2.000000
50%          3.000000
75%          6.000000
max        944.000000
Name: exchange_timestamp, dtype: float64


In [44]:
live_up['date'] = pd.to_datetime(live_up['exchange_timestamp'], unit='ms')
tes_up['date'] = pd.to_datetime(hist_up['exchange_timestamp'], unit='ms')
hist_up['date'] = pd.to_datetime(hist_up['exchange_timestamp'], unit='ms')

In [73]:
live_up[['exchange_timestamp', 'date', 'bid_size_3','bid_price_3','bid_size_2', 'bid_price_2','bid_size_1','bid_price_1', 'ask_price_1','ask_size_1','ask_price_2','ask_size_2','ask_price_3','ask_size_3']]

,exchange_timestamp,date,bid_size_3,bid_price_3,bid_size_2,bid_price_2,bid_size_1,bid_price_1,ask_price_1,ask_size_1,ask_price_2,ask_size_2,ask_price_3,ask_size_3
0,1771348804585,2026-02-17 17:20:04.585,508.46,0.55,412.15,0.56,5.00,0.57,0.58,6.00,0.59,122.99,0.60,237.50
2,1771348804677,2026-02-17 17:20:04.677,509.46,0.55,415.15,0.56,15.00,0.57,0.58,6.00,0.59,118.99,0.60,237.50
4,1771348804701,2026-02-17 17:20:04.701,509.46,0.55,415.15,0.56,12.00,0.57,0.58,6.00,0.59,118.99,0.60,242.50
6,1771348804716,2026-02-17 17:20:04.716,509.46,0.55,415.15,0.56,2.44,0.57,0.58,22.08,0.59,118.99,0.60,242.50
8,1771348804726,2026-02-17 17:20:04.726,514.46,0.55,415.15,0.56,22.44,0.57,0.58,16.47,0.59,118.99,0.60,242.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8730,1771349104057,2026-02-17 17:25:04.057,0.00,0.00,0.00,0.00,0.00,0.00,0.01,507772.15,0.02,22036.00,0.03,443.91
8732,1771349104441,2026-02-17 17:25:04.441,0.00,0.00,0.00,0.00,0.00,0.00,0.01,507767.15,0.02,22036.00,0.03,443.91
8886,1771349104441,2026-02-17 17:25:04.441,0.00,0.00,0.00,0.00,0.00,0.00,0.01,507767.15,0.02,22036.00,0.03,443.91
8734,1771349104523,2026-02-17 17:25:04.523,0.00,0.00,0.00,0.00,0.00,0.00,0.01,507748.27,0.02,22036.00,0.03,443.91


In [72]:
hist_up[['exchange_timestamp', 'date', 'bid_size_3','bid_price_3','bid_size_2', 'bid_price_2','bid_size_1','bid_price_1', 'ask_price_1','ask_size_1','ask_price_2','ask_size_2','ask_price_3','ask_size_3']]

,exchange_timestamp,date,bid_size_3,bid_price_3,bid_size_2,bid_price_2,bid_size_1,bid_price_1,ask_price_1,ask_size_1,ask_price_2,ask_size_2,ask_price_3,ask_size_3
0,1771348800080,2026-02-17 17:20:00.080,60.00,0.49,85.23,0.50,5.00,0.51,0.52,54.21,0.53,1411.76,0.54,34.99
3,1771348800104,2026-02-17 17:20:00.104,60.00,0.49,85.23,0.50,85.00,0.51,0.52,54.21,0.53,1411.76,0.54,34.99
5,1771348800330,2026-02-17 17:20:00.330,85.23,0.50,85.00,0.51,25.79,0.52,0.52,54.21,0.53,1411.76,0.54,34.99
6,1771348800430,2026-02-17 17:20:00.430,85.23,0.50,85.00,0.51,15.60,0.52,0.53,1411.76,0.54,34.99,0.55,62.98
8,1771348800433,2026-02-17 17:20:00.433,85.23,0.50,85.00,0.51,20.60,0.52,0.53,1411.76,0.54,34.99,0.55,62.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78331,1771349099532,2026-02-17 17:24:59.532,0.00,0.00,0.00,0.00,0.00,0.00,0.01,23247.50,0.02,426.37,0.03,283.91
78333,1771349099626,2026-02-17 17:24:59.626,0.00,0.00,0.00,0.00,0.00,0.00,0.01,23497.50,0.02,426.37,0.03,283.91
78335,1771349099718,2026-02-17 17:24:59.718,0.00,0.00,0.00,0.00,0.00,0.00,0.01,38497.50,0.02,426.37,0.03,283.91
78336,1771349099893,2026-02-17 17:24:59.893,0.00,0.00,0.00,0.00,0.00,0.00,0.01,38699.52,0.02,426.37,0.03,283.91


Index(['exchange_timestamp', 'received_timestamp', 'market', 'asset_id',
       'side', 'trade_price', 'size', 'best_bid', 'best_ask', 'mid_price',
       'spread', 'hash', 'date'],
      dtype='str')

In [75]:
tes_up[['exchange_timestamp', 'asset_id', 'side', 'size','trade_price', 'best_bid','best_ask']].loc[(tes_up['exchange_timestamp'] >= 1771348804579) & (tes_up['exchange_timestamp'] <= 1771348804679)]

,exchange_timestamp,asset_id,side,size,trade_price,best_bid,best_ask
0,1771348804579,1098611300385439941545533652037690501774261256...,BUY,508.46,0.55,0.57,0.59
3,1771348804580,1098611300385439941545533652037690501774261256...,SELL,6.00,0.58,0.57,0.58
5,1771348804581,1098611300385439941545533652037690501774261256...,SELL,122.99,0.59,0.57,0.58
7,1771348804583,1098611300385439941545533652037690501774261256...,SELL,609.00,0.62,0.57,0.58
8,1771348804584,1098611300385439941545533652037690501774261256...,BUY,558.73,0.52,0.57,0.58
10,1771348804585,1098611300385439941545533652037690501774261256...,BUY,370.22,0.46,0.57,0.58
13,1771348804586,1098611300385439941545533652037690501774261256...,SELL,627.00,0.64,0.57,0.58
15,1771348804588,1098611300385439941545533652037690501774261256...,SELL,614.00,0.62,0.57,0.58
16,1771348804591,1098611300385439941545533652037690501774261256...,BUY,285.10,0.32,0.57,0.58
18,1771348804593,1098611300385439941545533652037690501774261256...,BUY,10.00,0.57,0.57,0.58


In [79]:
slug   = "btc-updown-5m-1771348800"
trades = pd.read_parquet(f"data/trade_events/{slug}.parquet")
trades = trades.sort_values("exchange_timestamp").reset_index(drop=True)

asset_id = "109861130038543994154553365203769050177426125683071298510071991418247079830271"
t = trades[trades["asset_id"] == asset_id].reset_index(drop=True)

# Show the full sequence around timestamp 1771348804585-1771348804586
mask = (t["exchange_timestamp"] >= 1771348804580) & (t["exchange_timestamp"] <= 1771348804600)
print(t[mask][["exchange_timestamp", "side", "trade_price", 
               "best_bid", "best_ask", "mid_price", "size"]].to_string())

# Also check: does trade_price + paired_token_price = 1.0 for these rows?
# i.e. is the 0.46 actually the Down token price being reported on the Up token row?
print(f"\n0.46 + 0.54 = {0.46 + 0.54}")  # paired Down token price from row 10
print(f"0.64 + 0.36 = {0.64 + 0.36}")  # paired Down token price from row 13

    exchange_timestamp  side  trade_price  best_bid  best_ask  mid_price    size
1        1771348804580  SELL         0.58      0.57      0.58      0.575    6.00
2        1771348804581  SELL         0.59      0.57      0.58      0.575  122.99
3        1771348804583  SELL         0.62      0.57      0.58      0.575  609.00
4        1771348804584   BUY         0.52      0.57      0.58      0.575  558.73
5        1771348804585   BUY         0.46      0.57      0.58      0.575  370.22
6        1771348804586  SELL         0.64      0.57      0.58      0.575  627.00
7        1771348804588  SELL         0.62      0.57      0.58      0.575  614.00
8        1771348804591   BUY         0.32      0.57      0.58      0.575  285.10
9        1771348804593   BUY         0.57      0.57      0.58      0.575   10.00
10       1771348804597   BUY         0.49      0.57      0.58      0.575  476.23

0.46 + 0.54 = 1.0
0.64 + 0.36 = 1.0


In [80]:
slug   = "btc-updown-5m-1771348800"
trades = pd.read_parquet(f"data/trade_events/{slug}.parquet")
hist   = pd.read_parquet(f"data/telonex_book_snapshots/{slug}.parquet")

asset_id = "109861130038543994154553365203769050177426125683071298510071991418247079830271"

# The anomalous timestamps
bad_ts = [1771348804585, 1771348804591, 1771348804597]

hist_rows = hist[
    (hist["asset_id"] == asset_id) & 
    (hist["exchange_timestamp"].isin(bad_ts))
][["exchange_timestamp", "mid_price", "bid_price_1", "ask_price_1"]]

trade_rows = trades[
    (trades["asset_id"] == asset_id) & 
    (trades["exchange_timestamp"].isin(bad_ts))
][["exchange_timestamp", "side", "trade_price", "best_bid", "best_ask"]]

print("Trade events at anomalous timestamps:")
print(trade_rows.to_string())
print("\nTelonex book state at same timestamps:")
print(hist_rows.to_string())

Trade events at anomalous timestamps:
    exchange_timestamp side  trade_price  best_bid  best_ask
10       1771348804585  BUY         0.46      0.57      0.58
16       1771348804591  BUY         0.32      0.57      0.58
20       1771348804597  BUY         0.49      0.57      0.58

Telonex book state at same timestamps:
Empty DataFrame
Columns: [exchange_timestamp, mid_price, bid_price_1, ask_price_1]
Index: []


In [69]:
live_up[['exchange_timestamp', 'date', 'bid_size_3','bid_price_3','bid_size_2', 'bid_price_2','bid_size_1','bid_price_1', 'ask_price_1','ask_size_1','ask_price_2','ask_size_2','ask_price_3','ask_size_3']].loc[(live_up['exchange_timestamp'] >= 1771348804579) & (live_up['exchange_timestamp'] <= 1771348804679)]

,exchange_timestamp,date,bid_size_3,bid_price_3,bid_size_2,bid_price_2,bid_size_1,bid_price_1,ask_price_1,ask_size_1,ask_price_2,ask_size_2,ask_price_3,ask_size_3
0,1771348804585,2026-02-17 17:20:04.585,508.46,0.55,412.15,0.56,5.0,0.57,0.58,6.0,0.59,122.99,0.6,237.5
2,1771348804677,2026-02-17 17:20:04.677,509.46,0.55,415.15,0.56,15.0,0.57,0.58,6.0,0.59,118.99,0.6,237.5


In [76]:
slug   = "btc-updown-5m-1771348800"
trades = pd.read_parquet(f"data/trade_events/{slug}.parquet")
trades = trades.sort_values("exchange_timestamp").reset_index(drop=True)

# Find the suspicious sequence you're describing
# Look for large price deviations from mid
trades["fill_vs_mid"] = trades["trade_price"] - trades["mid_price"]

# Show rows where fill price deviates more than 5 cents from post-fill mid
suspicious = trades[trades["fill_vs_mid"].abs() > 0.05].copy()
print(f"Suspicious fills (>5c from post-fill mid): {len(suspicious):,}")
print(suspicious[["exchange_timestamp", "asset_id", "side", "trade_price", 
                   "best_bid", "best_ask", "mid_price", "fill_vs_mid", "size"]].head(20).to_string())

# Also show the surrounding context for the specific sequence you saw
# Find a sell followed quickly by a buy with large price swings
trades["ts_diff"] = trades["exchange_timestamp"].diff()
trades["price_diff"] = trades["trade_price"].diff()

rapid_reversals = trades[
    (trades["ts_diff"] < 10) &           # within 10ms
    (trades["price_diff"].abs() > 0.10)  # price moved >10c
]
print(f"\nRapid reversals (>10c move in <10ms): {len(rapid_reversals):,}")
print(rapid_reversals[["exchange_timestamp", "side", "trade_price", 
                        "mid_price", "fill_vs_mid", "size", "ts_diff", 
                        "price_diff"]].head(20).to_string())

Suspicious fills (>5c from post-fill mid): 58,242
    exchange_timestamp                                                                        asset_id  side  trade_price  best_bid  best_ask  mid_price  fill_vs_mid    size
8        1771348804584   26853942177931886201757336755289682872943860044150497825685268669651915309817  SELL         0.48      0.42      0.43      0.425        0.055  558.73
9        1771348804584  109861130038543994154553365203769050177426125683071298510071991418247079830271   BUY         0.52      0.57      0.58      0.575       -0.055  558.73
10       1771348804585   26853942177931886201757336755289682872943860044150497825685268669651915309817  SELL         0.54      0.42      0.43      0.425        0.115  370.22
11       1771348804585  109861130038543994154553365203769050177426125683071298510071991418247079830271   BUY         0.46      0.57      0.58      0.575       -0.115  370.22
12       1771348804586  10986113003854399415455336520376905017742612568307129851

In [78]:
slug   = "btc-updown-5m-1771348800"
trades = pd.read_parquet(f"data/trade_events/{slug}.parquet")
trades = trades.sort_values(["exchange_timestamp", "size"]).reset_index(drop=True)

# Pair rows by matching timestamp + size across the two asset_ids
asset_ids = trades["asset_id"].unique()
#assert len(asset_ids) == 2, f"Expected 2 asset_ids, got {len(asset_ids)}"

t0 = trades[trades["asset_id"] == asset_ids[0]].set_index(["exchange_timestamp", "size"])
t1 = trades[trades["asset_id"] == asset_ids[1]].set_index(["exchange_timestamp", "size"])

paired = t0.join(t1, lsuffix="_up", rsuffix="_dn", how="inner")
paired["price_sum"] = paired["trade_price_up"] + paired["trade_price_dn"]

print(f"Total trades     : {len(trades):,}")
print(f"Paired matches   : {len(paired):,}  ({2*len(paired)/len(trades):.1%} of rows paired)")
print(f"\nPrice sum (should be ~1.0):")
print(paired["price_sum"].describe())
print(f"\nNon-unity price sums (>0.001 from 1.0): "
      f"{(paired['price_sum'] - 1.0).abs().gt(0.001).sum():,}")

Total trades     : 139,046
Paired matches   : 71,187  (102.4% of rows paired)

Price sum (should be ~1.0):
count    71187.000000
mean         1.000000
std          0.025244
min          0.400000
25%          1.000000
50%          1.000000
75%          1.000000
max          1.600000
Name: price_sum, dtype: float64

Non-unity price sums (>0.001 from 1.0): 1,192


## Check to see if Telonex orderbook snapshots are actually interleaving trade events as well. This might account for the order-of-magnitude mismatch in data rows

In [16]:
"""
Verify Telonex = live book_snapshots + trade_events interleaved.

Logic:
  - Load live book_snapshots and trade_events for a matching slug
  - For each trade event, reconstruct a synthetic book row by forward-filling
    the last known book state, then overwriting mid_price/spread/book_imbalance
    with values computed from the trade event's best_bid/best_ask
  - Merge synthetic rows with real book snapshot rows, sort by timestamp
  - Compare the resulting interleaved stream to Telonex on:
      1. Row count / density
      2. Timestamp overlap
      3. Value deltas at matching timestamps

Usage:
    python interleave_comparison.py
    python interleave_comparison.py --slug btc-updown-5m-1771348200
"""

import argparse
from pathlib import Path

import numpy as np
import pandas as pd

# ---------------------------------------------------------------------------
# Config
# ---------------------------------------------------------------------------

LIVE_BOOK_DIR  = Path("data/book_snapshots")
LIVE_TRADE_DIR = Path("data/trade_events")
HIST_DIR       = Path("data/telonex_book_snapshots")
TIMESTAMP_COL  = "exchange_timestamp"
N_LEVELS       = 5   # Telonex only has 5 levels

NUMERIC_COLS = (
    ["mid_price", "spread", "book_imbalance"]
    + [f"bid_price_{i}" for i in range(1, N_LEVELS + 1)]
    + [f"bid_size_{i}"  for i in range(1, N_LEVELS + 1)]
    + [f"ask_price_{i}" for i in range(1, N_LEVELS + 1)]
    + [f"ask_size_{i}"  for i in range(1, N_LEVELS + 1)]
)

# ---------------------------------------------------------------------------
# Helpers
# ---------------------------------------------------------------------------

def find_common_slug() -> str | None:
    live_slugs = {p.stem for p in LIVE_BOOK_DIR.glob("btc-updown-5m-*.parquet")}
    hist_slugs = {p.stem for p in HIST_DIR.glob("btc-updown-5m-*.parquet")}
    common = live_slugs & hist_slugs
    if not common:
        return None
    # Prefer a slug that also has a trade_events file
    for slug in sorted(common):
        if (LIVE_TRADE_DIR / f"{slug}.parquet").exists():
            return slug
    return sorted(common)[0]


def load_live_books(slug: str) -> pd.DataFrame:
    path = LIVE_BOOK_DIR / f"{slug}.parquet"
    df = pd.read_parquet(path).sort_values(TIMESTAMP_COL).reset_index(drop=True)
    return df


def load_live_trades(slug: str) -> pd.DataFrame | None:
    path = LIVE_TRADE_DIR / f"{slug}.parquet"
    if not path.exists():
        print(f"  WARNING: No trade file found at {path}")
        return None
    df = pd.read_parquet(path).sort_values(TIMESTAMP_COL).reset_index(drop=True)
    return df


def load_hist(slug: str) -> pd.DataFrame:
    path = HIST_DIR / f"{slug}.parquet"
    df = pd.read_parquet(path).sort_values(TIMESTAMP_COL).reset_index(drop=True)
    return df


def trades_to_book_rows(trades: pd.DataFrame, books: pd.DataFrame) -> pd.DataFrame:
    """
    For each trade event, synthesize a book row by:
      1. Forward-filling the last known book snapshot state (bid/ask levels)
      2. Overwriting mid_price, spread, book_imbalance from the trade's
         best_bid / best_ask fields (which reflect the book state post-trade)

    This is the hypothesis: Telonex emits one row per event (book update OR
    trade), so interleaving trades into the book stream should approximate it.
    """
    if trades is None or trades.empty:
        return pd.DataFrame()

    # We'll build synthetic book rows for each trade event.
    # The book level columns come from the last snapshot before the trade.
    book_cols = [c for c in NUMERIC_COLS if c in books.columns]

    # Index books by timestamp for ffill lookup
    books_ts = books.set_index(TIMESTAMP_COL).sort_index()

    rows = []
    for _, trade in trades.iterrows():
        ts        = trade[TIMESTAMP_COL]
        asset_id  = trade["asset_id"]

        # Find the last book snapshot at or before this timestamp for this asset
        asset_books = books_ts[books_ts["asset_id"] == asset_id]
        prior = asset_books.loc[:ts]

        if prior.empty:
            # No prior snapshot — skip this trade (can't reconstruct book state)
            continue

        last_book = prior.iloc[-1].copy()

        # Overwrite derived fields from the trade's reported best bid/ask
        best_bid = trade.get("best_bid", np.nan)
        best_ask = trade.get("best_ask", np.nan)

        if pd.notna(best_bid) and pd.notna(best_ask) and best_bid > 0 and best_ask > 0:
            mid   = (best_bid + best_ask) / 2
            sprd  = best_ask - best_bid

            # Recompute book_imbalance from top-5 levels of the last snapshot,
            # using the trade's best_bid/ask to update level 1
            bid_sizes = [last_book.get(f"bid_size_{i}", 0.0) for i in range(1, N_LEVELS + 1)]
            ask_sizes = [last_book.get(f"ask_size_{i}", 0.0) for i in range(1, N_LEVELS + 1)]
            total_bid = sum(bid_sizes)
            total_ask = sum(ask_sizes)
            denom     = total_bid + total_ask
            imbalance = (total_bid - total_ask) / denom if denom else 0.0
        else:
            mid       = last_book.get("mid_price", np.nan)
            sprd      = last_book.get("spread", np.nan)
            imbalance = last_book.get("book_imbalance", np.nan)

        row = {
            TIMESTAMP_COL:   ts,
            "asset_id":       asset_id,
            "mid_price":      mid,
            "spread":         sprd,
            "book_imbalance": imbalance,
            "source":         "trade",
        }
        # Carry forward book levels from last snapshot
        for col in book_cols:
            if col not in ("mid_price", "spread", "book_imbalance"):
                row[col] = last_book.get(col, np.nan)

        rows.append(row)

    return pd.DataFrame(rows)


def interleave(books: pd.DataFrame, trade_rows: pd.DataFrame) -> pd.DataFrame:
    """Merge book snapshots and synthetic trade rows, sort by timestamp."""
    books_tagged  = books.copy()
    books_tagged["source"] = "book"

    combined = pd.concat([books_tagged, trade_rows], ignore_index=True)
    combined = combined.sort_values([TIMESTAMP_COL, "source"]).reset_index(drop=True)
    return combined


def interval_stats(df: pd.DataFrame, asset_id: str, label: str):
    sub = df[df["asset_id"] == asset_id].sort_values(TIMESTAMP_COL)
    diffs = sub[TIMESTAMP_COL].diff().dropna()
    print(f"\n  {label} — timestamp intervals (ms) for asset_id={asset_id[:12]}...")
    print(f"    count  : {len(diffs):,}")
    print(f"    mean   : {diffs.mean():.2f}")
    print(f"    median : {diffs.median():.2f}")
    print(f"    std    : {diffs.std():.2f}")
    print(f"    min    : {diffs.min():.2f}")
    print(f"    p25    : {diffs.quantile(0.25):.2f}")
    print(f"    p75    : {diffs.quantile(0.75):.2f}")
    print(f"    max    : {diffs.max():.2f}")


# ---------------------------------------------------------------------------
# Main
# ---------------------------------------------------------------------------

def main(slug: str | None = None):
    slug = slug or find_common_slug()
    if slug is None:
        print("ERROR: No slug found in both directories.")
        return

    print(f"\n{'='*65}")
    print(f"  Interleave comparison for: {slug}")
    print(f"{'='*65}")

    books  = load_live_books(slug)
    trades = load_live_trades(slug)
    hist   = load_hist(slug)

    print(f"\n  Live book rows  : {len(books):,}")
    print(f"  Live trade rows : {len(trades):,}" if trades is not None else "  Live trade rows : N/A")
    print(f"  Telonex rows    : {len(hist):,}")

    # -----------------------------------------------------------------------
    # 1. Baseline interval stats (before interleaving)
    # -----------------------------------------------------------------------
    # Use the first asset_id seen in live books for all comparisons
    live_asset_id = books["asset_id"].iloc[0]
    hist_token_df = hist[hist["token_label"] == "Up"] if "token_label" in hist.columns else hist

    print(f"\n--- Baseline intervals (before interleaving) ---")
    interval_stats(books,         live_asset_id, "Live books only")
    if trades is not None:
        interval_stats(trades,    live_asset_id, "Live trades only")
    interval_stats(hist_token_df, hist_token_df["asset_id"].iloc[0] if "asset_id" in hist_token_df.columns else "", "Telonex")

    # -----------------------------------------------------------------------
    # 2. Build interleaved stream
    # -----------------------------------------------------------------------
    print(f"\n--- Building interleaved stream ---")
    trade_rows   = trades_to_book_rows(trades, books)
    interleaved  = interleave(books, trade_rows)

    print(f"  Book rows          : {len(books):,}")
    print(f"  Synthetic trade rows: {len(trade_rows):,}")
    print(f"  Interleaved total  : {len(interleaved):,}")
    print(f"  Telonex total      : {len(hist):,}")
    ratio = len(interleaved) / len(hist) if len(hist) else 0
    print(f"  Interleaved / Telonex ratio: {ratio:.2%}  (1.0 = perfect match)")

    interval_stats(interleaved, live_asset_id, "Interleaved (books + trades)")

    # -----------------------------------------------------------------------
    # 3. Timestamp overlap: interleaved vs Telonex
    # -----------------------------------------------------------------------
    print(f"\n--- Timestamp overlap ---")

    interleaved_ts = set(interleaved[interleaved["asset_id"] == live_asset_id][TIMESTAMP_COL])
    hist_asset_id  = hist_token_df["asset_id"].iloc[0] if "asset_id" in hist_token_df.columns else None
    hist_ts        = set(hist_token_df[TIMESTAMP_COL]) if hist_asset_id else set()

    common_ts = interleaved_ts & hist_ts
    print(f"  Interleaved timestamps : {len(interleaved_ts):,}")
    print(f"  Telonex timestamps     : {len(hist_ts):,}")
    print(f"  Common timestamps      : {len(common_ts):,}  "
          f"({100*len(common_ts)/max(len(hist_ts),1):.1f}% of Telonex)")

    # -----------------------------------------------------------------------
    # 4. Value deltas at common timestamps
    # -----------------------------------------------------------------------
    if not common_ts:
        print("\n  No common timestamps — cannot compute deltas.")
        print("  This confirms the two sources have different event clocks.")
        return

    compare_cols = [c for c in NUMERIC_COLS
                    if c in interleaved.columns and c in hist_token_df.columns]

    inter_aligned = (
        interleaved[interleaved["asset_id"] == live_asset_id]
        .set_index(TIMESTAMP_COL)
        .loc[list(common_ts), compare_cols]
        .sort_index()
    )
    hist_aligned = (
        hist_token_df
        .set_index(TIMESTAMP_COL)
        .loc[list(common_ts), compare_cols]
        .sort_index()
    )

    delta = (inter_aligned - hist_aligned).abs()

    print(f"\n--- Value deltas at {len(common_ts):,} common timestamps ---")
    print(f"\n  {'Column':<25} {'max_delta':>12} {'mean_delta':>12} {'nonzero_rows':>14}")
    print(f"  {'-'*65}")

    for col in compare_cols:
        col_delta = delta[col].dropna()
        max_d  = col_delta.max()
        mean_d = col_delta.mean()
        nz     = (col_delta > 0).sum()
        flag   = " ⚠" if nz > 0 else ""
        print(f"  {col:<25} {max_d:>12.6f} {mean_d:>12.6f} {nz:>14,}{flag}")

    # -----------------------------------------------------------------------
    # 5. Timestamps only in Telonex (not in live at all)
    # -----------------------------------------------------------------------
    only_hist_ts = hist_ts - interleaved_ts
    print(f"\n--- Timestamps only in Telonex (not in live + trades) ---")
    print(f"  Count: {len(only_hist_ts):,}  ({100*len(only_hist_ts)/max(len(hist_ts),1):.1f}% of Telonex)")

    if only_hist_ts:
        sample_ts = sorted(only_hist_ts)[:5]
        print(f"  Sample: {sample_ts}")
        print(f"  These timestamps have no corresponding live event.")
        print(f"  If this count is high, Telonex has events your feed never received.")

    print(f"\n{'='*65}")
    print("  INTERPRETATION GUIDE")
    print(f"{'='*65}")
    print("""
  Interleaved / Telonex ratio ~1.0
      → Hypothesis confirmed. Telonex = books + trades interleaved.
        The training/inference gap is manageable.

  Ratio << 1.0, low timestamp overlap
      → Telonex has events from a different source (e.g. order book diffs,
        internal order events). The two feeds are fundamentally different.
        Training on Telonex will produce a model that sees a richer world
        than live inference can provide.

  Ratio ~1.0 but large value deltas
      → Row counts match but computed values differ. Check bid sort order,
        imbalance computation, or level-depth truncation differences.

  Many timestamps only in Telonex
      → Telonex received WebSocket messages your ingest missed (drops,
        reconnects, latency). Consider this a data quality ceiling on live.
""")


if __name__ == "__main__":
    main("btc-updown-5m-1771348800")


  Interleave comparison for: btc-updown-5m-1771348800

  Live book rows  : 8,890
  Live trade rows : 139,046
  Telonex rows    : 78,340

--- Baseline intervals (before interleaving) ---

  Live books only — timestamp intervals (ms) for asset_id=109861130038...
    count  : 4,395
    mean   : 68.25
    median : 33.00
    std    : 116.30
    min    : 0.00
    p25    : 16.00
    p75    : 76.00
    max    : 2291.00

  Live trades only — timestamp intervals (ms) for asset_id=109861130038...
    count  : 69,242
    mean   : 4.33
    median : 2.00
    std    : 13.30
    min    : 0.00
    p25    : 1.00
    p75    : 3.00
    max    : 496.00

  Telonex — timestamp intervals (ms) for asset_id=109861130038...
    count  : 39,169
    mean   : 7.66
    median : 3.00
    std    : 22.45
    min    : 1.00
    p25    : 2.00
    p75    : 6.00
    max    : 944.00

--- Building interleaved stream ---
  Book rows          : 8,890
  Synthetic trade rows: 139,036
  Interleaved total  : 147,926
  Telonex tota

In [17]:
import pandas as pd
from pathlib import Path

slug = "btc-updown-5m-1771348800"

books  = pd.read_parquet(f"data/book_snapshots/{slug}.parquet")
trades = pd.read_parquet(f"data/trade_events/{slug}.parquet")
hist   = pd.read_parquet(f"data/telonex_book_snapshots/{slug}.parquet")

# --- Test 1: how many exact timestamp collisions exist in live data? ---
# (same timestamp, same asset_id appearing in both books and trades)
book_ts  = set(zip(books["exchange_timestamp"],  books["asset_id"]))
trade_ts = set(zip(trades["exchange_timestamp"], trades["asset_id"]))
collisions = book_ts & trade_ts
print(f"Exact timestamp collisions (book + trade same ms): {len(collisions):,}")
# If this is non-trivial (hundreds+), dedup explains the row count difference

# --- Test 2: nearest-timestamp match instead of exact ---
# For each Telonex timestamp, find the closest interleaved timestamp
# and measure the distribution of offsets
up_asset = hist[hist["token_label"] == "Up"]["asset_id"].iloc[0]
hist_up  = hist[hist["token_label"] == "Up"].sort_values("exchange_timestamp")

live_all = pd.concat([
    books[books["asset_id"] == up_asset][["exchange_timestamp"]].assign(source="book"),
    trades[trades["asset_id"] == up_asset][["exchange_timestamp"]].assign(source="trade"),
]).sort_values("exchange_timestamp").reset_index(drop=True)

hist_ts_arr  = hist_up["exchange_timestamp"].values
live_ts_arr  = live_all["exchange_timestamp"].values

# For each hist timestamp, find nearest live timestamp
import numpy as np
idx     = np.searchsorted(live_ts_arr, hist_ts_arr)
idx     = np.clip(idx, 0, len(live_ts_arr) - 1)
offsets = np.abs(hist_ts_arr - live_ts_arr[idx])

print(f"\nNearest-timestamp offset (ms) — Telonex vs live interleaved:")
print(pd.Series(offsets).describe())
print(f"Within 1ms  : {(offsets <= 1).mean():.1%}")
print(f"Within 5ms  : {(offsets <= 5).mean():.1%}")
print(f"Within 10ms : {(offsets <= 10).mean():.1%}")

Exact timestamp collisions (book + trade same ms): 8,736

Nearest-timestamp offset (ms) — Telonex vs live interleaved:
count    39170.000000
mean        22.554072
std        234.420589
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max       4499.000000
dtype: float64
Within 1ms  : 98.6%
Within 5ms  : 98.6%
Within 10ms : 98.7%


In [18]:
# Isolate the mismatched population
import numpy as np

idx     = np.searchsorted(live_ts_arr, hist_ts_arr)
idx     = np.clip(idx, 0, len(live_ts_arr) - 1)
offsets = np.abs(hist_ts_arr - live_ts_arr[idx])

# Split into matched vs unmatched
threshold = 5  # ms
matched   = hist_up[offsets <= threshold].copy()
unmatched = hist_up[offsets >  threshold].copy()
unmatched["offset_ms"] = offsets[offsets > threshold]

print(f"Matched   : {len(matched):,}  ({len(matched)/len(hist_up):.1%})")
print(f"Unmatched : {len(unmatched):,}  ({len(unmatched)/len(hist_up):.1%})")

# Where in the market window do unmatched rows cluster?
# Extract market open timestamp from slug
slug_ts   = int(slug.split("-")[-1]) * 1000  # slug unix seconds → ms
market_end = slug_ts + 300_000

unmatched["seconds_into_market"] = (unmatched["exchange_timestamp"] - slug_ts) / 1000
print("\nUnmatched rows — position in market window (seconds):")
print(unmatched["seconds_into_market"].describe())

# Are they clustered at specific times, or spread evenly?
print("\nDistribution across 30s buckets:")
bins = pd.cut(unmatched["seconds_into_market"], bins=10)
print(unmatched.groupby(bins, observed=True).size())

# What do the unmatched rows look like — are they distinct price levels?
print("\nUnmatched rows — mid_price distribution:")
print(unmatched["mid_price"].describe())
print("\nMatched rows — mid_price distribution:")
print(matched["mid_price"].describe())

Matched   : 38,639  (98.6%)
Unmatched : 531  (1.4%)

Unmatched rows — position in market window (seconds):
count    531.000000
mean       3.311881
std        9.277767
min        0.080000
25%        1.795500
50%        3.194000
75%        3.953000
max      215.057000
Name: seconds_into_market, dtype: float64

Distribution across 30s buckets:
seconds_into_market
(-0.135, 21.578]      530
(193.559, 215.057]      1
dtype: int64

Unmatched rows — mid_price distribution:
count    531.000000
mean       0.544266
std        0.031439
min        0.075000
25%        0.525000
50%        0.525000
75%        0.565000
max        0.585000
Name: mid_price, dtype: float64

Matched rows — mid_price distribution:
count    38639.000000
mean         0.356192
std          0.172974
min          0.005000
25%          0.175000
50%          0.425000
75%          0.485000
max          0.605000
Name: mid_price, dtype: float64


In [19]:
import numpy as np
import pandas as pd
from pathlib import Path

LIVE_BOOK_DIR  = Path("data/book_snapshots")
LIVE_TRADE_DIR = Path("data/trade_events")
HIST_DIR       = Path("data/telonex_book_snapshots")

live_slugs = {p.stem for p in LIVE_BOOK_DIR.glob("btc-updown-5m-*.parquet")}
hist_slugs = {p.stem for p in HIST_DIR.glob("btc-updown-5m-*.parquet")}
common = sorted(live_slugs & hist_slugs)

print(f"Checking {len(common)} slugs...\n")

results = []
for slug in common[:20]:  # sample 20
    try:
        books  = pd.read_parquet(LIVE_BOOK_DIR  / f"{slug}.parquet")
        trades = pd.read_parquet(LIVE_TRADE_DIR / f"{slug}.parquet") \
                 if (LIVE_TRADE_DIR / f"{slug}.parquet").exists() else pd.DataFrame()
        hist   = pd.read_parquet(HIST_DIR / f"{slug}.parquet")

        slug_ts   = int(slug.split("-")[-1]) * 1000
        up_asset  = hist[hist["token_label"] == "Up"]["asset_id"].iloc[0]
        hist_up   = hist[hist["token_label"] == "Up"].sort_values("exchange_timestamp")

        live_ts = np.sort(pd.concat([
            books[books["asset_id"] == up_asset][["exchange_timestamp"]],
            trades[trades["asset_id"] == up_asset][["exchange_timestamp"]] if not trades.empty else pd.DataFrame(columns=["exchange_timestamp"]),
        ])["exchange_timestamp"].values)

        hist_ts = hist_up["exchange_timestamp"].values
        idx     = np.clip(np.searchsorted(live_ts, hist_ts), 0, len(live_ts) - 1)
        offsets = np.abs(hist_ts - live_ts[idx]) if len(live_ts) else hist_ts

        unmatched_mask = offsets > 5
        unmatched_secs = (hist_ts[unmatched_mask] - slug_ts) / 1000

        results.append({
            "slug":               slug,
            "hist_rows":          len(hist_up),
            "live_rows":          len(live_ts),
            "match_rate":         (~unmatched_mask).mean(),
            "unmatched_mean_sec": unmatched_secs.mean() if len(unmatched_secs) else np.nan,
            "unmatched_p75_sec":  np.percentile(unmatched_secs, 75) if len(unmatched_secs) else np.nan,
        })
    except Exception as e:
        print(f"  {slug}: ERROR — {e}")

df = pd.DataFrame(results)
print(df.to_string(index=False))
print(f"\nMedian match rate : {df['match_rate'].median():.1%}")
print(f"Median unmatched mean position : {df['unmatched_mean_sec'].median():.1f}s into market")
print(f"Median unmatched p75 position  : {df['unmatched_p75_sec'].median():.1f}s into market")

Checking 1585 slugs...

  btc-updown-5m-1771350900: ERROR — Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.
  btc-updown-5m-1771351500: ERROR — Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.
  btc-updown-5m-1771352100: ERROR — Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.
                    slug  hist_rows  live_rows  match_rate  unmatched_mean_sec  unmatched_p75_sec
btc-updown-5m-1771346400      34798       2653    0.027042          108.745437          166.45700
btc-updown-5m-1771346700      49828      67877    0.746026           34.206882           50.77750
btc-updown-5m-1771347000      43046      82484    0.993100            1.894572            2.49400
btc-updown-5m-1771347300     

In [20]:
slug  = "btc-updown-5m-1771348800"
books = pd.read_parquet(f"data/book_snapshots/{slug}.parquet")
hist  = pd.read_parquet(f"data/telonex_book_snapshots/{slug}.parquet")

# Compare raw size magnitudes
print("Live bid_size_1 distribution:")
print(books["bid_size_1"].describe())

print("\nTelonex bid_size_1 distribution (Up token):")
print(hist[hist["token_label"]=="Up"]["bid_size_1"].describe())

Live bid_size_1 distribution:
count      8890.000000
mean       4221.360717
std       41770.377153
min           0.000000
25%          39.872500
50%         111.700000
75%         232.000000
max      523770.780000
Name: bid_size_1, dtype: float64

Telonex bid_size_1 distribution (Up token):
count    39170.000000
mean       217.859717
std        691.042554
min          0.000000
25%         29.000000
50%         84.880000
75%        187.280000
max      15075.840000
Name: bid_size_1, dtype: float64


In [21]:
slug  = "btc-updown-5m-1771348800"
books = pd.read_parquet(f"data/book_snapshots/{slug}.parquet")
hist  = pd.read_parquet(f"data/telonex_book_snapshots/{slug}.parquet")
hist_up = hist[hist["token_label"] == "Up"]

# Look at the full size distribution excluding outliers
import numpy as np

for label, sizes in [("Live", books["bid_size_1"]), ("Telonex", hist_up["bid_size_1"])]:
    p99 = sizes.quantile(0.99)
    print(f"{label} bid_size_1 — excluding top 1%:")
    print(sizes[sizes <= p99].describe())
    print(f"  p99={p99:.1f},  p99.9={sizes.quantile(0.999):.1f},  max={sizes.max():.1f}\n")

# Are the giant live values transient (appear once) or persistent?
large_live = books[books["bid_size_1"] > 10000].sort_values("exchange_timestamp")
print(f"Live rows with bid_size_1 > 10,000: {len(large_live)}")
if len(large_live):
    print(large_live[["exchange_timestamp", "bid_size_1", "bid_price_1", "mid_price"]].head(10))
    # Check if consecutive — are they persistent orders or one-off spikes?
    ts_diffs = large_live["exchange_timestamp"].diff()
    print(f"\nTime between large-size rows (ms):")
    print(ts_diffs.describe())

Live bid_size_1 — excluding top 1%:
count     8801.000000
mean       619.597051
std       2800.121926
min          0.000000
25%         39.210000
50%        110.300000
75%        225.240000
max      28301.710000
Name: bid_size_1, dtype: float64
  p99=28303.9,  p99.9=522275.4,  max=523770.8

Telonex bid_size_1 — excluding top 1%:
count    38784.000000
mean       172.782335
std        327.045785
min          0.000000
25%         28.660000
50%         82.120000
75%        183.000000
max       2522.160000
Name: bid_size_1, dtype: float64
  p99=2522.2,  p99.9=14010.1,  max=15075.8

Live rows with bid_size_1 > 10,000: 273
     exchange_timestamp  bid_size_1  bid_price_1  mid_price
44        1771348805272   523770.78         0.99      0.495
46        1771348805321   523759.20         0.99      0.495
98        1771348806025   523599.20         0.99      0.495
134       1771348806778   522553.49         0.99      0.495
156       1771348807090   522376.22         0.99      0.495
160       177134

In [22]:
slug  = "btc-updown-5m-1771348800"
books = pd.read_parquet(f"data/book_snapshots/{slug}.parquet")

# Look at all 10 bid levels for the large-size rows
large = books[books["bid_size_1"] > 10000].copy()

bid_price_cols = [f"bid_price_{i}" for i in range(1, 11)]
bid_size_cols  = [f"bid_size_{i}"  for i in range(1, 11)]

print("Large bid_size_1 rows — all bid levels:")
print(large[bid_price_cols + bid_size_cols].head(3).to_string())

# Also check: is the 0.99 bid always level 1, or does it appear at other levels?
print("\nBid price distributions across levels:")
for i in range(1, 6):
    col = f"bid_price_{i}"
    print(f"  {col}: mean={books[col].mean():.4f}, "
          f"min={books[col].min():.4f}, max={books[col].max():.4f}")

# What does the normal book look like vs the large-size rows?
normal = books[books["bid_size_1"] <= 10000]
print(f"\nNormal rows bid_price_1: mean={normal['bid_price_1'].mean():.4f}, "
      f"max={normal['bid_price_1'].max():.4f}")
print(f"Large rows  bid_price_1: mean={large['bid_price_1'].mean():.4f}, "
      f"min={large['bid_price_1'].min():.4f}")

Large bid_size_1 rows — all bid levels:
    bid_price_1  bid_price_2  bid_price_3  bid_price_4  bid_price_5  bid_price_6  bid_price_7  bid_price_8  bid_price_9  bid_price_10  bid_size_1  bid_size_2  bid_size_3  bid_size_4  bid_size_5  bid_size_6  bid_size_7  bid_size_8  bid_size_9  bid_size_10
44         0.99         0.98         0.97         0.96         0.95         0.94         0.93         0.92         0.91           0.9   523770.78     21923.0    10209.74     17832.8    24020.04     8575.32    23669.77    20646.98    14543.88     11389.98
46         0.99         0.98         0.97         0.96         0.95         0.94         0.93         0.92         0.91           0.9   523759.20     21923.0    10209.74     17832.8    24020.04     8575.32    23669.77    20646.98    14543.88     11389.98
98         0.99         0.98         0.97         0.96         0.95         0.94         0.93         0.92         0.91           0.9   523599.20     21923.0    10209.74     17832.8    24020.04  